In [124]:
import requests
import os
import json
import pandas as pd

load_dotenv()

#base URL and the endpoint for authentication
base_url = 'https://trytako.com'
auth_endpoint = '/auth-token/'

# Your credentials (replace with your actual credentials)
payload = {
    'username': os.getenv("TAKO_USERNAME"),
    'password': os.getenv("PASSWORD")
}

# Send the POST request to get the authentication token
response = requests.post(base_url + auth_endpoint, data=payload)
token = response.json()['token']

In [138]:
collections_endpoint= '/api/v1/list_collections/'

headers = {
    'accept': 'application/json',
    'Authorization': 'Token ' + token
}

collect = requests.get(base_url + collections_endpoint, headers=headers)
collect_dict = collect.json()
count = 0
while count < 10:
    print(collect_dict['collections'][count])
    print('\n')
    count+=1

{"name":"30 Year Fixed Mortgage Average","collection_description":"This is the Average 30-Year Fixed Mortgage Rate in the United States reported on a weekly basis.","allowed_aggregations":["3-month-trailing-average","year-over-year","month-over-month","percentage-change-since-start"]}


{"name":"Annual CO2 Emissions","collection_description":"This is annual CO2 emissions by fuel source (petroleum, natural gas, coal, all) and sector (commercial, industrial, residential, transportation) for each State in the United States (e.g. Colorado).","allowed_aggregations":["percentage-change-since-start"]}


{"name":"Avg. Interest Rates on US Treasury Securities","collection_description":"The Average Interest Rates on U.S. Treasury Securities dataset provides average interest rates on U.S. Treasury securities on a monthly basis. Its primary purpose is to show the average interest rate on a variety of marketable and non-marketable Treasury securities. Marketable securities consist of Treasury Bills

In [49]:
#testing search endpoint
search_endpoint = '/api/v1/search/'
data = {
    "query": "Israel vs Iran military spending and GDP over time",
    "start_time": "2010-01-01",
    "fast_filter": False
}
search = requests.post(base_url + search_endpoint, headers=headers, data=data)
search.json()

{'url': 'https://trytako.com/embed/pzmSnEqYw0V2XvPqL9QN/',
 'iframe': '<iframe width="100%" src="https://trytako.com/embed/pzmSnEqYw0V2XvPqL9QN/" scrolling="no" frameborder="0"></iframe><script type="text/javascript">',
 'chart_description': 'Country Economic Indicators: Annual GDP - Current USD from 2010-01-01 to 2023-01-01 for Israel - most recent value, from 2023-01-01 is 509901495702.10 delta from 2010-01-01T00:00:00+00:00 to 2023-01-01T00:00:00+00:00 is 271537403404.08 (113.92%), Iran - most recent value, from 2023-01-01 is 401504514718.72 delta from 2010-01-01T00:00:00+00:00 to 2023-01-01T00:00:00+00:00 is -85303102157.98 (-17.52%)Military Metrics: Annual Military Spending - Current from 2010-01-01 to 2023-01-01 for Military Metrics - most recent value, from 2023-01-01 is 27498527856.85 delta from 2010-01-01T00:00:00+00:00 to 2023-01-01T00:00:00+00:00 is 13623316261.15 (98.18%), Military Metrics - most recent value, from 2023-01-01 is 10283084401.75 delta from 2010-01-01T00:00:00

In [50]:
#testing search endpoint
data = {
    "query": "countries with the highest military spending as a % of their national GDP",
    "start_time": "2010-01-01",
    "fast_filter": False
}
search = requests.post(base_url + search_endpoint, headers=headers, data=data)
search.json()

{'url': 'https://trytako.com/embed/QlbcRJYIB7lHRZaZVlAo/',
 'iframe': '<iframe width="100%" src="https://trytako.com/embed/QlbcRJYIB7lHRZaZVlAo/" scrolling="no" frameborder="0"></iframe><script type="text/javascript">',
 'chart_description': 'Military Metrics: Annual Military Spending as a Share of Real GDP from 2014-01-01 to 2023-01-01 for Military Metrics - most recent value, from 2023-01-01 is 3.49 delta from 2014-01-01T00:00:00+00:00 to 2023-01-01T00:00:00+00:00 is -0.23 (-0.06%) delta from 2014-01-01T00:00:00+00:00 to 2023-01-01T00:00:00+00:00 is -0.23 (-0.06%) -',
 'title': 'United States Military Spending as Share of GDP',
 'source_url': 'https://trytako.com/',
 'specificity': '100',
 'pub_id': 'QlbcRJYIB7lHRZaZVlAo',
 'visualization_type': 'timeseries'}

In [56]:
#function for quering vizQL data service
def send(query):
    url = os.getenv('VDS_URL')
    payload = json.dumps({
        "connection": {
            "tableauServerName": os.getenv('TABLEAU_DOMAIN'),
            "siteId": os.getenv('SITE_NAME'),
            "datasource": os.getenv('DATA_SOURCE')
        },
        "query": query
    })

    headers = {
    'Credential-Key': os.getenv('PAT_NAME'),
    'Credential-value': os.getenv('PAT_SECRET'),
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response data
        #print("Response Data:")
        #print(response.json())
        data = response.json()['data']
        # Create a pandas DataFrame from the JSON data
        df = pd.DataFrame(data)

        # Display the first few rows of the DataFrame
        #print("Table view of data from the public REST API:")
        #print(df.head())
        #print(df.all())
        return df
        #display(df.head())
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        print(response.text)

In [127]:
#simple VDS query for retrieving sum of sales per year
vds_payload = {'columns': [{'columnName': 'Order Date', "function": "DATE_YEAR",
            "sortPriority":1}, {'columnName': 'Sales', 'function': 'SUM'}]}
output = send(vds_payload)
output

,YEAR(Order Date),SUM(Sales)
0,2021,494040.2121
1,2022,472993.0310
2,2023,613933.5800
3,2024,745567.5312


In [116]:
series_data = {
    "series": [
        {
            "records": [
                {
                    "x": pd.to_datetime(row['YEAR(Order Date)']).strftime('%Y-%m-%dT%H:%M:%S'),
                    "y": row['SUM(Sales)']
                }
                for _, row in output.iterrows()
            ]
        }
    ]
}

# Now you can pass 'series_data' through the API
print(series_data['series'])


[{'records': [{'x': '2021-01-01T00:00:00', 'y': 494040.2121000005}, {'x': '2022-01-01T00:00:00', 'y': 472993.03100000083}, {'x': '2023-01-01T00:00:00', 'y': 613933.5799999994}, {'x': '2024-01-01T00:00:00', 'y': 745567.5311999985}]}]


In [135]:
def construct_time_series(df, date_col, value_col, date_format='%Y-%m-%dT%H:%M:%S'):
    """
    Converts a DataFrame into a list of dictionaries suitable for the records parameter in the Tako render endpoint

    Parameters:
    - dataframe (pd.DataFrame): The input DataFrame.
    - date_col (str): The name of the column containing date values.
    - value_col (str): The name of the column containing numeric values.
    - date_format (str): The desired date format (default is ISO 8601).

    Returns:
    - records_list (list): A list of dictionaries with keys 'x' and 'y'.
    """
    records_list = []
    for _, row in output.iterrows():
        # Convert the date to datetime object
        date_obj = pd.to_datetime(row[date_col])

        # Format the date to ISO 8601
        iso_date = date_obj.strftime(date_format)

        # Build the dictionary
        record = { "x": iso_date, "y": row[value_col]}

        # Append to the list
        records_list.append(record)

    return records_list

In [136]:
#invoke the construct_time_series function for the superstore output dataframe
records = construct_time_series(df=output,date_col='YEAR(Order Date)', value_col='SUM(Sales)')
records

[{'x': '2021-01-01T00:00:00', 'y': 494040.2121000005},
 {'x': '2022-01-01T00:00:00', 'y': 472993.03100000083},
 {'x': '2023-01-01T00:00:00', 'y': 613933.5799999994},
 {'x': '2024-01-01T00:00:00', 'y': 745567.5311999985}]

In [137]:
#testing tako render endpoint with superstore data queried through VDS
render_endpoint = '/api/v1/render_data/'
data = {
    "title": "Superstore Sales per Year",
    "subtitle": "",
    "series": [
        {
            "records": records,
            "label": "test label",
            "y_units": "custom units",
            "viz_type": "line"
        },
    ],
    "save_preview_image": False
}

requests.post(base_url + render_endpoint, headers=headers, json=data).json()

{'url': 'https://trytako.com/embed/FAp_qMIkGRv5rZrcWOGX/',
 'iframe': '<iframe width="100%" src="https://trytako.com/embed/FAp_qMIkGRv5rZrcWOGX/" scrolling="no" frameborder="0"></iframe><script type="text/javascript">',
 'chart_description': 'Custom chart',
 'title': 'Superstore Sales per Year',
 'source_url': 'https://trytako.com/',
 'specificity': '100',
 'pub_id': 'FAp_qMIkGRv5rZrcWOGX'}